## Stardog Database Connection Setup
 - Import required libraries (os, dotenv, stardog)
 - Load environment variables for Stardog credentials
 - Define StardogClient class for database connection and queries
 - Implement query and update methods with error handling


In [29]:
import os
from dotenv import load_dotenv
import stardog

load_dotenv()

# Stardog configuration
STARDOG_ENDPOINT = os.getenv("STARDOG_ENDPOINT")
STARDOG_DATABASE = os.getenv("STARDOG_DATABASE")
STARDOG_USERNAME = os.getenv("STARDOG_USERNAME")
STARDOG_PASSWORD = os.getenv("STARDOG_PASSWORD")

class StardogClient:
    def __init__(self, endpoint, database, username, password):
        self.endpoint = endpoint
        self.database = database
        self.connection_details = {
            'endpoint': endpoint,
            'username': username,
            'password': password
        }
        self.database = database

    def query(self, sparql_query, use_reasoning=True):
        """Execute a SPARQL query against Stardog"""
        try:
            with stardog.Connection(self.database, **self.connection_details) as conn:
                # Execute the query with reasoning if enabled
                result = conn.select(sparql_query, reasoning=use_reasoning)
                return result
        except Exception as e:
            print(f"Error executing SPARQL query: {e}")
            return None

    def update(self, sparql_update):
        """Execute a SPARQL UPDATE query"""
        try:
            with stardog.Connection(self.database, **self.connection_details) as conn:
                conn.begin()
                conn.update(sparql_update)
                conn.commit()
                return True
        except Exception as e:
            print(f"Error executing SPARQL update: {e}")
            return False

    def get_schema(self):
        """Get the ontology schema from the knowledge graph, including classes, properties, and their domains and ranges."""
        schema_details_query = """
        SELECT ?s ?p ?o
        FROM <tag:stardog:api:context:schema>
        WHERE {
            ?s ?p ?o
        }
        """
        schema_result = self.query(schema_details_query, use_reasoning=False)
        if schema_result and schema_result.get('results', {}).get('bindings'):
            triples = schema_result['results']['bindings']

            classes = set()
            object_properties = {}
            datatype_properties = {}

            ontology_base = 'https://a.in/sales/ontology#'
            def short_name(uri):
                return uri[len(ontology_base):] if uri.startswith(ontology_base) else uri

            for triple in triples:
                subject = triple['s']['value']
                predicate = triple['p']['value']
                obj = triple['o']['value']

                # Identify classes
                if predicate == 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type':
                    if obj == 'http://www.w3.org/2002/07/owl#Class':
                        cls = short_name(subject)
                        if cls != 'https://a.in/sales/ontology':
                            classes.add(cls)
                    elif obj == 'http://www.w3.org/2002/07/owl#ObjectProperty':
                        op = short_name(subject)
                        if op not in object_properties:
                            object_properties[op] = {'domain': None, 'range': None}
                    elif obj == 'http://www.w3.org/2002/07/owl#DatatypeProperty':
                        dp = short_name(subject)
                        if dp not in datatype_properties:
                            datatype_properties[dp] = {'domain': None, 'range': None}

                # Identify domain/range
                if predicate == 'http://www.w3.org/2000/01/rdf-schema#domain':
                    prop = short_name(subject)
                    val = short_name(obj)
                    if prop in object_properties:
                        object_properties[prop]['domain'] = val
                    elif prop in datatype_properties:
                        datatype_properties[prop]['domain'] = val
                if predicate == 'http://www.w3.org/2000/01/rdf-schema#range':
                    prop = short_name(subject)
                    val = short_name(obj)
                    if prop in object_properties:
                        object_properties[prop]['range'] = val
                    elif prop in datatype_properties:
                        datatype_properties[prop]['range'] = val

            return {
                'classes': sorted(classes),
                'object_properties': object_properties,
                'datatype_properties': datatype_properties,
                'raw_result': schema_result
            }
        return None

# Initialize Stardog client
stardog_client = StardogClient(STARDOG_ENDPOINT, STARDOG_DATABASE, STARDOG_USERNAME, STARDOG_PASSWORD)

# Sample ontology prefixes for e-commerce domain
PREFIXES = """
PREFIX ex: <https://a.in/sales/ontology#>
PREFIX data: <https://a.in/sales/data#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX schema: <http://schema.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
"""

print("Stardog KG client initialized successfully!")

Stardog KG client initialized successfully!


## Schema Exploration
- Initializing Stardog client to connect to the knowledge graph
- Retrieving and displaying the schema information including:
  - Classes (Product, Customer, Order etc.)
  - Object properties (relationships between classes)
  - Datatype properties (attributes of classes)
- Helps understand the structure and organization of the e-commerce ontology


In [ ]:
stardog_client = StardogClient(STARDOG_ENDPOINT, STARDOG_DATABASE, STARDOG_USERNAME, STARDOG_PASSWORD)
print("Stardog database:", STARDOG_DATABASE)
display(stardog_client.get_schema())

Stardog endpoint: https://sd-682b640b.stardog.cloud:5820
Stardog database: Sales_360
Stardog username: testing
Stardog password: KnowledgeGraph


{'classes': ['https://sales.org/sales/ontology#Customer',
  'https://sales.org/sales/ontology#CustomerAddress',
  'https://sales.org/sales/ontology#CustomerContact',
  'https://sales.org/sales/ontology#Inventory',
  'https://sales.org/sales/ontology#Order',
  'https://sales.org/sales/ontology#Product',
  'https://sales.org/sales/ontology#ProductBrand',
  'https://sales.org/sales/ontology#ProductClass',
  'https://sales.org/sales/ontology#ProductDetail',
  'https://sales.org/sales/ontology#ProductFamily',
  'https://sales.org/sales/ontology#ProductLine',
  'https://sales.org/sales/ontology#ProductType',
  'https://sales.org/sales/ontology#Sale',
  'https://sales.org/sales/ontology#SalesDiscountTax',
  'https://sales.org/sales/ontology#SalesFulfillment',
  'https://sales.org/sales/ontology#SalesPayment',
  'https://sales.org/sales/ontology#Supplier'],
 'object_properties': {'https://sales.org/sales/ontology#hasContact': {'domain': 'https://sales.org/sales/ontology#Customer',
   'range': 